In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os
import pandas as pd
from pandas.api.types import is_integer_dtype, is_float_dtype

In [32]:
import os
import pandas as pd
from pandas.api.types import is_integer_dtype, is_float_dtype

BASE = "/content/drive/MyDrive/Pushshift"

# фінальні файли
clean_path = f"{BASE}/final_dataset_clean_all_v2.csv"
small_path = f"{BASE}/final_dataset_small_all_v2.csv"

topics = [
    "anxiety",
    "depression",
    "offmychest",
    "covid19",
    "coronavirus",
    "covid19_support",
    "mentalhealthsupport"
]

clean_path, small_path

('/content/drive/MyDrive/Pushshift/final_dataset_clean_all_v2.csv',
 '/content/drive/MyDrive/Pushshift/final_dataset_small_all_v2.csv')

In [33]:
def clean_chunk_all(chunk: pd.DataFrame) -> pd.DataFrame:
    # 1. created_utc → datetime
    if 'created_utc' not in chunk.columns:
        # немає часу — такий шматок нам не допоможе
        return pd.DataFrame()

    # якщо timestamp числовий
    if is_integer_dtype(chunk['created_utc']) or is_float_dtype(chunk['created_utc']):
        chunk['created_utc'] = pd.to_datetime(
            chunk['created_utc'],
            unit='s',
            utc=True,
            errors='coerce'
        )
    else:
        # якщо вже str
        chunk['created_utc'] = pd.to_datetime(
            chunk['created_utc'],
            utc=True,
            errors='coerce'
        )

    # прибираємо рядки без дати
    chunk = chunk.dropna(subset=['created_utc'])
    if chunk.empty:
        return pd.DataFrame()

    # 2. month
    chunk['month'] = chunk['created_utc'].dt.to_period('M').astype(str)

    # 3. вибір текстової колонки
    text_col = None
    for c in ['body', 'selftext', 'text']:
        if c in chunk.columns:
            text_col = c
            break

    if text_col is None:
        return pd.DataFrame()

    chunk['text'] = chunk[text_col].astype('string')
    chunk['text_len'] = chunk['text'].str.len()

    # 4. score → Int64
    if 'score' in chunk.columns:
        chunk['score'] = pd.to_numeric(chunk['score'], errors='coerce')
    else:
        chunk['score'] = pd.NA
    chunk['score'] = chunk['score'].astype('Int64')

    # 5. subreddit, author, id — як string
    if 'subreddit' in chunk.columns:
        chunk['subreddit'] = chunk['subreddit'].astype('string')
    if 'author' in chunk.columns:
        chunk['author'] = chunk['author'].astype('string')
    if 'id' in chunk.columns:
        chunk['id'] = chunk['id'].astype('string')

    # 6. залишаємо потрібні колонки
    keep = ['id', 'author', 'subreddit', 'created_utc',
            'month', 'text', 'text_len', 'score']

    existing = [c for c in keep if c in chunk.columns]
    chunk = chunk[existing]

    return chunk

In [34]:
for p in [clean_path, small_path]:
    if os.path.exists(p):
        os.remove(p)
        print("Removed:", p)

In [36]:
chunksize = 200_000
first_clean = True
first_small = True

total_in = 0
total_out = 0
total_small = 0

for topic in topics:
    topic_dir = os.path.join(BASE, topic)
    if not os.path.isdir(topic_dir):
        print(f"Directory not found: {topic_dir}, skipping")
        continue

    print(f"\n Topic: {topic}")
    files = sorted(f for f in os.listdir(topic_dir) if f.endswith(".csv"))
    print(f"Found {len(files)} files in {topic_dir}")

    for fname in files:
        fpath = os.path.join(topic_dir, fname)
        print(f"  Processing file: {fname}")

        reader = pd.read_csv(
            fpath,
            chunksize=chunksize,
            engine="python",
            on_bad_lines="skip"
        )

        for i, raw_chunk in enumerate(reader):
            total_in += len(raw_chunk)

            cleaned = clean_chunk_all(raw_chunk)
            if cleaned.empty:
                print(f"    Chunk {i}: empty after cleaning, skipped")
                continue

            total_out += len(cleaned)


            mode = "w" if first_clean else "a"
            cleaned.to_csv(clean_path, mode=mode, header=first_clean, index=False)
            first_clean = False


            sample_frac = 0.05
            sample = cleaned.sample(frac=sample_frac, random_state=42) if len(cleaned) > 0 else cleaned
            if not sample.empty:
                mode_s = "w" if first_small else "a"
                sample.to_csv(small_path, mode=mode_s, header=first_small, index=False)
                first_small = False
                total_small += len(sample)

            print(f"    Chunk {i}: raw={len(raw_chunk)}, cleaned={len(cleaned)}, total_cleaned={total_out}, total_small={total_small}")

print("Total raw rows seen:", total_in)
print("Total cleaned rows:", total_out)
print("Total sample rows:", total_small)
print("Clean file saved to:", clean_path)
print("Small file saved to:", small_path)


 Topic: anxiety
Found 8 files in /content/drive/MyDrive/Pushshift/anxiety
  Processing file: RC_2020-02.csv


/tmp/ipython-input-3845327170.py:29: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  chunk['month'] = chunk['created_utc'].dt.to_period('M').astype(str)


    Chunk 0: raw=26402, cleaned=26402, total_cleaned=26402, total_small=1320
  Processing file: RC_2020-03.csv


/tmp/ipython-input-3845327170.py:29: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  chunk['month'] = chunk['created_utc'].dt.to_period('M').astype(str)


    Chunk 0: raw=44439, cleaned=44439, total_cleaned=70841, total_small=3542
  Processing file: RC_2020-04.csv


...

/tmp/ipython-input-3845327170.py:29: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  chunk['month'] = chunk['created_utc'].dt.to_period('M').astype(str)


    Chunk 0: raw=1206, cleaned=1206, total_cleaned=7016789, total_small=350839
  Processing file: RC_2020-08.csv


/tmp/ipython-input-3845327170.py:29: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  chunk['month'] = chunk['created_utc'].dt.to_period('M').astype(str)


    Chunk 0: raw=1232, cleaned=1232, total_cleaned=7018021, total_small=350901
  Processing file: RC_2020-09.csv
    Chunk 0: raw=1295, cleaned=1295, total_cleaned=7019316, total_small=350966
Total raw rows seen: 7019316
Total cleaned rows: 7019316
Total sample rows: 350966
Clean file saved to: /content/drive/MyDrive/Pushshift/final_dataset_clean_all_v2.csv
Small file saved to: /content/drive/MyDrive/Pushshift/final_dataset_small_all_v2.csv


/tmp/ipython-input-3845327170.py:29: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  chunk['month'] = chunk['created_utc'].dt.to_period('M').astype(str)


In [37]:
df_small = pd.read_csv(small_path)

df_small['created_utc'] = pd.to_datetime(df_small['created_utc'], utc=True, errors='coerce')
df_small['month'] = df_small['created_utc'].dt.to_period('M').astype(str)

print("Shape:", df_small.shape)
print("\nMonths:")
print(df_small['month'].value_counts().sort_index())

print("\nSubreddits:")
print(df_small['subreddit'].value_counts())

/tmp/ipython-input-3936753600.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_small['month'] = df_small['created_utc'].dt.to_period('M').astype(str)


Shape: (350966, 8)

Months:
month
2020-02     32096
2020-03     14243
2020-04    101263
2020-05     57749
2020-06     38145
2020-07     50743
2020-08     32435
2020-09     24292
Name: count, dtype: int64

Subreddits:
subreddit
Coronavirus            262395
offmychest              29131
depression              24604
COVID19                 17086
Anxiety                 13820
COVID19_support          3541
MentalHealthSupport       389
Name: count, dtype: int64
